In [1]:
from funlib.geometry import Coordinate
from dacapo.experiments.architectures import CNNectomeUNetConfig
from dacapo.experiments.tasks.distance_task_config import DistanceTaskConfig

import torch

input_voxel_size = (16, 16, 16)
output_voxel_size = (8, 8, 8)

channels = [
    "ecs",  # extra cellular space
    "plasma_membrane",
    "mito",
    "mito_membrane",
    "vesicle",
    "vesicle_membrane",
    "mvb",  # endosomes
    "mvb_membrane",
    "er",
    "er_membrane",
    "eres",
    "nucleus",
    "microtubules",
    "microtubules_out",
]

architecture_config = CNNectomeUNetConfig(
    name="CellMapArchitecture",
    input_shape=Coordinate(
        216, 216, 216
    ),  # can be changed
    eval_shape_increase=Coordinate(
        72, 72, 72
    ),  # can be changed
    fmaps_in=1,
    num_fmaps=12,
    fmaps_out=72,
    fmap_inc_factor=6,
    downsample_factors=[(2, 2, 2), (3, 3, 3), (3, 3, 3)],
    constant_upsample=True,
    upsample_factors=[(2, 2, 2)],
)

task_config = DistanceTaskConfig(
    name="DistancePrediction",
    # important
    channels=channels,
    scale_factor=50,  # target = tanh(distance / scale)
    # training
    mask_distances=True,
    # evaluation
    clip_distance=50,
    tol_distance=10,
)

# create backbone from config
architecture = architecture_config.architecture_type(architecture_config)

# initialize task from config
task = task_config.task_type(task_config)

# adding final layers/activations to create the model
model = task.create_model(architecture)


path_to_weights = "/nrs/cellmap/pattonw/crop_num_experiment/scratch_distances_many_all_many_8nm_upsample-unet_default__2/checkpoints/mito__f1_score"
weights = torch.load(path_to_weights, map_location="cuda")
model.load_state_dict(weights["model"])
model.to("cuda")
model.eval()

/groups/scicompsoft/home/ackermand/miniconda3/envs/neuroglancer_interactive_inference/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/groups/scicompsoft/home/ackermand/miniconda3/envs/neuroglancer_interactive_inference/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


Model(
  (architecture): CNNectomeUNet(
    (unet): Sequential(
      (0): CNNectomeUNetModule(
        (l_conv): ModuleList(
          (0): ConvPass(
            (conv_pass): Sequential(
              (0): Conv3d(1, 12, kernel_size=(3, 3, 3), stride=(1, 1, 1))
              (1): ReLU()
              (2): Conv3d(12, 12, kernel_size=(3, 3, 3), stride=(1, 1, 1))
              (3): ReLU()
            )
          )
          (1): ConvPass(
            (conv_pass): Sequential(
              (0): Conv3d(12, 72, kernel_size=(3, 3, 3), stride=(1, 1, 1))
              (1): ReLU()
              (2): Conv3d(72, 72, kernel_size=(3, 3, 3), stride=(1, 1, 1))
              (3): ReLU()
            )
          )
          (2): ConvPass(
            (conv_pass): Sequential(
              (0): Conv3d(72, 432, kernel_size=(3, 3, 3), stride=(1, 1, 1))
              (1): ReLU()
              (2): Conv3d(432, 432, kernel_size=(3, 3, 3), stride=(1, 1, 1))
              (3): ReLU()
            )
          )
  

In [5]:
# taken from https://github.com/google/neuroglancer/blob/master/python/examples/interactive_inference.py
"""Example of displaying interactive image-to-image "inference" results.
shift+mousedown0 triggers the inference result to be computed for the patch
centered around the mouse position, and then displayed in neuroglancer.
In this example, the inference result is actually just a distance transform
computed from the ground truth segmentation, but in actual use the inference
result may be computed using SciPy, Tensorflow, PyTorch, etc.
The cloudvolume library (https://github.com/seung-lab/cloud-volume) is used to
retrieve patches of the ground truth volume.
The zarr library is used to represent the sparse in-memory array containing the
computed inference results that are displayed in neuroglancer.
"""

import argparse
import time

import neuroglancer
import neuroglancer.cli
import cloudvolume
import zarr
import numpy as np
import scipy.ndimage
import tensorstore as ts
import socket


class InteractiveInference(object):
    def __init__(self):
        neuroglancer.set_server_bind_address(
            bind_address=socket.gethostbyname(socket.gethostname())
        )
        viewer = self.viewer = neuroglancer.Viewer()
        viewer.actions.add("inference", self._do_inference)

        # self.raw_data = ts.open({
        # 'driver':   
        #          'n5',
        # 'kvstore':{
        #     'driver': 'http',
        #     'base_url': 'https://janelia-cosem-datasets.s3.amazonaws.com/jrc_mus-liver/jrc_mus-liver.n5/em/fibsem-uint8/s1',
        #    }}).result()
        group = zarr.open(zarr.N5FSStore('s3://janelia-cosem-datasets/jrc_mus-liver/jrc_mus-liver.n5/', anon=True)) # access the root of the n5 container
        self.raw_s1 = group['em/fibsem-uint8/s1'] # s0 is the the full-resolution data for this particular volume
        self.raw_s0 = group['em/fibsem-uint8/s0'] # s0 is the the full-resolution data for this particular volume
        # Create a 3-d view (remove singleton 'channel' dimension):

        self.dimensions = neuroglancer.CoordinateSpace(
                     names=["x", "y", "z"],
                     units="nm",
                     scales=self.raw_s0.attrs['transform']['scale'][::-1],
        )


        with viewer.config_state.txn() as s:
            s.input_event_bindings.data_view["shift+mousedown0"] = "inference"

        with viewer.txn() as s:
            s.layers["raw"] = neuroglancer.ImageLayer(
                source="n5://s3://janelia-cosem-datasets/jrc_mus-liver/jrc_mus-liver.n5/em/fibsem-uint8",
                shader="""#uicontrol invlerp normalized(range=[105, 182], window=[0, 255])\n
                #uicontrol int invertColormap slider(min=0, max=1, step=1, default=0)\n
        #uicontrol vec3 color color(default="white")
        float inverter(float val, int invert) {return 0.5 + ((2.0 * (-float(invert) + 0.5)) * (val - 0.5));}\n
          void main() {\n
          emitRGB(color * inverter(normalized(), invertColormap));
        }"""
            )
        
            self.inf_results = []
            self.inf_volumes = []
            for organelle in channels:
                self.inf_results.append( zarr.zeros(
                    self.raw_s0.shape[::-1], chunks=(64, 64, 64), dtype=np.uint8
                ))
                self.inf_volumes.append(neuroglancer.LocalVolume(
                    data=self.inf_results[-1], dimensions=self.dimensions
                ))
                s.layers[organelle] = neuroglancer.ImageLayer(source=self.inf_volumes[-1],
                    shader="#uicontrol invlerp normalized \nvoid main() {\n\temitRGB(vec3(normalized(),0, 0));\n}",
                    shaderControls = {
                            "normalized": {"range": [127, 127]}
                        },
                )
                s.layers[organelle].visible = organelle=="mito"
    #             s.layers["ground_truth"] = neuroglancer.SegmentationLayer(
    #                 source="precomputed://gs://neuroglancer-public-data/flyem_fib-25/ground_truth",
    #             )
    #             s.layers["ground_truth"].visible = False
    #             s.layers["inference"] = neuroglancer.ImageLayer(
    #                 source=self.inf_volume,
    #                 shader="""
    # void main() {
    #   float v = toNormalized(getDataValue(0));
    #   vec4 rgba = vec4(0,0,0,0);
    #   if (v != 0.0) {
    #     rgba = vec4(colormapJet(v), 1.0);
    #   }
    #   emitRGBA(rgba);
    # }
    # """,
    #             )

    def _do_inference(self, action_state):
        s0_pos = action_state.mouse_voxel_coordinates
        if s0_pos is None:
            return

        s1_pos = s0_pos[::-1]//2
        patch_size = np.array((216,) * 3, np.int64)
        spos = s1_pos - patch_size //2
        epos = spos + patch_size
        slice_expr = np.s_[
            int(spos[0]) : int(epos[0]),
            int(spos[1]) : int(epos[1]),
            int(spos[2]) : int(epos[2]),
        ]

        raw_s1 = self.raw_s1[slice_expr]/255.0
        raw_s1 = np.reshape(raw_s1, (1,1,216,216,216))
        outputs = model(torch.from_numpy(raw_s1).float().to("cuda"))
        outputs = (outputs.cpu().detach().numpy())
        outputs[outputs<-1] = -1
        outputs[outputs>1] = 1
        outputs = outputs*127.5+127.5
        patch_size = np.array((68,)*3, np.int64)

        spos = s0_pos - patch_size //2
        epos = spos + patch_size
        slice_expr = np.s_[
            int(spos[0]) : int(epos[0]),
            int(spos[1]) : int(epos[1]),
            int(spos[2]) : int(epos[2]),
        ]
        patch_size = np.array((68,) * 3, np.int64)
        
        for i,channel in enumerate(channels):
            self.inf_results[i][slice_expr] = np.transpose(outputs[0][i].astype(np.uint8))
            self.inf_volumes[i].invalidate()

inf = InteractiveInference()
print(inf.viewer)



http://10.150.100.248:42109/v/52c8e71b5b19385e4263f69241e6f173f2e57b9d/


In [1]:
import neuroglancer
import numpy as np
import zarr
import socket
from dask.distributed import Client

client = Client(threads_per_worker=2, n_workers=1)
client.cluster.scale(20)
dashboard_link = client.cluster.dashboard_link
print(dashboard_link.replace("127.0.0.1",socket.gethostbyname(socket.gethostname())))

def add_dask_layer(state):
    """Adds a lazily-computed data source backed by dask."""
    # https://docs.dask.org/en/latest/array-creation.html#using-dask-delayed
    import dask
    import dask.array

    def make_array(k):
        print("Computing k=%d" % (k,))
        return np.random.randint(low=0, high=255, size=(256, 256, 256), dtype=np.uint8)

    x_max = 12747
    y_max = 12728
    z_max = 8932
    lazy_make_array = dask.delayed(make_array, pure=True)
    count = 0
    shape = 68*4
    arrays = [None]*((int(np.ceil(x_max/shape))+1) * (int(np.ceil(y_max/shape))+1) * (int(np.ceil(z_max/shape))+1))
    for x in range(0,x_max,shape):
        print(x)
        for y in range(0,y_max,shape):
            for z in range(0,z_max,shape):
                arrays[count]= dask.array.from_delayed(lazy_make_array(count), dtype=np.uint8, shape=(256,256,256))
                count+=1
    arrays[count:]=[]
    # lazy_chunks[count:]=[]
    # sample = lazy_chunks[
    #     0
    # ].compute()  # load the first chunk (assume rest are same shape/dtype)
    # arrays = [
    #     dask.array.from_delayed(lazy_chunk, dtype=sample.dtype, shape=sample.shape)
    #     for lazy_chunk in lazy_chunks
    # ]
    group = zarr.open(zarr.N5FSStore('s3://janelia-cosem-datasets/jrc_mus-liver/jrc_mus-liver.n5/', anon=True)) # access the root of the n5 container
    raw_s1 = group['em/fibsem-uint8/s1'] # s0 is the the full-resolution data for this particular volume
    x = dask.array.concatenate(arrays)
    state.layers["dask"] = neuroglancer.ImageLayer(source = neuroglancer.LocalVolume(x))#dask.array.from_array(raw_s1, chunks=(64,64,64))))

neuroglancer.set_server_bind_address(
    bind_address=socket.gethostbyname(socket.gethostname())
)
viewer = neuroglancer.Viewer()
with viewer.txn() as s:
    add_dask_layer(s)
print(viewer)


2022-10-25 00:12:21,359 - distributed.diskutils - INFO - Found stale lock file and directory '/groups/cellmap/cellmap/ackermand/Programming/neuroglancer_interactive_inference/dask-worker-space/worker-xjn5s12d', purging
2022-10-25 00:12:21,366 - distributed.diskutils - INFO - Found stale lock file and directory '/groups/cellmap/cellmap/ackermand/Programming/neuroglancer_interactive_inference/dask-worker-space/worker-lf9j1iij', purging
2022-10-25 00:12:21,373 - distributed.diskutils - INFO - Found stale lock file and directory '/groups/cellmap/cellmap/ackermand/Programming/neuroglancer_interactive_inference/dask-worker-space/worker-cvp2cs47', purging
2022-10-25 00:12:21,379 - distributed.diskutils - INFO - Found stale lock file and directory '/groups/cellmap/cellmap/ackermand/Programming/neuroglancer_interactive_inference/dask-worker-space/worker-vufswzxt', purging
2022-10-25 00:12:21,386 - distributed.diskutils - INFO - Found stale lock file and directory '/groups/cellmap/cellmap/ackerm

http://10.150.100.248:8787/status
0
272
544
816
1088
1360
1632
1904
2176
2448
2720
2992
3264
3536
3808
4080
4352
4624
4896
5168
5440
5712
5984
6256
6528
6800
7072
7344
7616
7888
8160
8432
8704
8976
9248
9520
9792
10064
10336
10608
10880
11152
11424
11696
11968
12240
12512
http://10.150.100.248:34087/v/d93acb89a122eed1a427131c2f54de6c401ec975/


Computing k=36448
Computing k=36449Computing k=36447

Computing k=36448
Computing k=36448
Computing k=36448
Computing k=36449
Computing k=36449
Computing k=36448
Computing k=36448
Computing k=36449
Computing k=36448
Computing k=36447
Computing k=36449
Computing k=36448
Computing k=36447
Computing k=36449
Computing k=36447
Computing k=36449
Computing k=36447
Computing k=36447
Computing k=36447
Computing k=36448
Computing k=36448
Computing k=36448
Computing k=36448
Computing k=36448
Computing k=36448
Computing k=36448
Computing k=36448
Computing k=36448
Computing k=36448Computing k=36448

Computing k=36448
Computing k=36448
Computing k=36448
Computing k=36448
Computing k=36448
Computing k=36448
Computing k=36448
Computing k=36449
Computing k=36448Computing k=36448

Computing k=36449Computing k=36449

Computing k=36448
Computing k=36448
Computing k=36448
Computing k=36449
Computing k=36449Computing k=36448
Computing k=36448

Computing k=36448
Computing k=36448
Computing k=36448
Computing 

2022-10-25 00:17:23,171 - distributed.utils_perf - WARNING - full garbage collections took 27% CPU time recently (threshold: 10%)
2022-10-25 00:17:26,838 - distributed.utils_perf - WARNING - full garbage collections took 27% CPU time recently (threshold: 10%)


Computing k=36448Computing k=36448

Computing k=36449
Computing k=36449Computing k=36447



...

2022-10-25 00:32:55,968 - distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
2022-10-25 00:32:57,782 - distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
2022-10-25 00:33:01,836 - distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
2022-10-25 00:33:05,017 - distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
2022-10-25 00:33:07,672 - distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)


Computing k=36445
Computing k=36445
Computing k=36447Computing k=36445

Computing k=36445


2022-10-25 00:33:14,414 - distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
2022-10-25 00:33:16,047 - distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
2022-10-25 00:33:20,458 - distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
2022-10-25 00:33:25,800 - distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)


Computing k=36445Computing k=36445
Computing k=36445

Computing k=36445


: 

In [15]:
count

NameError: name 'count' is not defined